In [2]:
import requests
from bs4 import BeautifulSoup
import lxml.html
from selenium import webdriver
import time
import pandas as pd
import re
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def scrape_more_beer(pages):
    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    url = 'https://www.beeradvocate.com/community/login/'
    driver.get(url)
    
    login_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dd//input[@name ="login"]')))
    login_element.send_keys('HaniNguyen_1803')

    pw_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit"]//dd//ul//li[@id = "ctrl_pageLogin_registered_Disabler"]//input[@name ="password"]')))
    pw_element.send_keys('scrapebeer1234')

    page_click = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit submitUnit"]//dd//input[@type ="submit"]')))
    page_click.click()
    
    mnf = []
    pro_link = []
    product = []
    for page_num in tqdm(range(0,pages,50)):
        new_url = "https://www.beeradvocate.com/user/beers/?start=%d&&ba=HaniNguyen_1803&order=dateD&view=W" %(page_num)
        req = driver.get(new_url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        page_source = driver.page_source
        soup = lxml.html.fromstring(page_source)
        mnf_i = soup.xpath('//td[@valign ="top"]/a/text()')[::2]
        pro_link_i = soup.xpath('//td[@valign ="top"]/a/@href')[::3]
        product_i = soup.xpath('//td[@valign ="top"]/a/b/text()')
        mnf.append(mnf_i)
        pro_link.append(pro_link_i)
        product.append(product_i)
        
    pro_link_rm_ns_ls = [ i for m in pro_link for i in m]
    mnf_rm_ns_ls = [ i for m in mnf for i in m]
    product_rm_ns_ls = [ i for m in product for i in m]
    mnf = pd.DataFrame({'mnf': mnf_rm_ns_ls, "pro_link": pro_link_rm_ns_ls, "product": product_rm_ns_ls})
    return mnf

In [34]:
mnfac = scrape_more_beer(100)

100%|██████████| 2/2 [00:08<00:00,  4.06s/it]


In [40]:
mnfac

,mnf,pro_link,product
0,Beijing Yanjing Beer Group Corporation,/beer/profile/713/2124/,Yanjing Beer
1,Wychwood Brewery Company Ltd,/beer/profile/160/508425/,Hobgoblin Ruby 4.5%
2,Widmer Brothers Brewing Company,/beer/profile/8/80302/,Omission Pale Ale (Gluten-Free)
3,Brouwerij Westmalle,/beer/profile/208/646/,Westmalle Trappist Tripel
4,Wernesgrüner Brauerei AG,/beer/profile/3966/8653/,Wernesgruner
...,...,...,...
91,Swinkels Family Brewers,/beer/profile/498/1422/,Bavaria Beer
92,Bard's Tale Beer Company,/beer/profile/11034/47565/,Bard's Gold
93,Baltika Breweries,/beer/profile/401/4695/,Baltika #7 Export
94,Asahi Breweries Ltd,/beer/profile/716/2779/,Asahi Super Dry


In [11]:
def webstite_scrape_p2 (beer_link):

    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    url = 'https://www.beeradvocate.com/community/login/'
    driver.get(url)
    
    login_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dd//input[@name ="login"]')))
    login_element.send_keys('HaniNguyen_1803')

    pw_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit"]//dd//ul//li[@id = "ctrl_pageLogin_registered_Disabler"]//input[@name ="password"]')))
    pw_element.send_keys('scrapebeer1234')

    page_click = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit submitUnit"]//dd//input[@type ="submit"]')))
    page_click.click()
    
    mnf =[]
    style =[]
    alc =[]
    ratings = [] #number of ratings
    avg_rating = [] 
    number_of_reviews = []
    
    for link in tqdm(beer_link): 
        print(link)
        new_url = "https://www.beeradvocate.com/"+link
        print(new_url)
        driver.get(new_url)

        time.sleep(5)
        page_source = driver.page_source
        soup = lxml.html.fromstring(page_source)
        
        mnf_i = soup.xpath('//dd[@class = "beerstats"]/a[@class = "Tooltip"]/text()')
        style_i = soup.xpath('//dd[@class = "beerstats"]/a[@class = "Tooltip"]/b/text()')
        alc_i = soup.xpath('//dd[@class = "beerstats"]/span[@class = "Tooltip"]/b/text()')
        ratings_i= soup.xpath('//dd[@class = "beerstats"]/span[@class = "ba-ratings Tooltip"]/text()')
        
        avg_i = soup.xpath('//dd[@class = "beerstats"]/b/span[@class = "ba-ravg Tooltip"]/text()')
        number_of_reviews_i = soup.xpath('//dd[@class = "beerstats"]/span[@class = "ba-reviews Tooltip"]/text()')
        
        mnf.append( mnf_i)
        style.append(style_i)
        alc.append(alc_i)
        ratings.append(ratings_i)
        avg_rating.append(avg_i)
        number_of_reviews.append(number_of_reviews_i)

    return mnf, style, alc, ratings, avg_rating, number_of_reviews

In [12]:
mnf, style, alc, ratings, avg_rating, number_of_reviews =webstite_scrape_p2(mnf.pro_link)


  0%|          | 0/96 [00:00<?, ?it/s]

/beer/profile/713/2124/
https://www.beeradvocate.com//beer/profile/713/2124/


  1%|          | 1/96 [00:07<11:17,  7.14s/it]

/beer/profile/160/508425/
https://www.beeradvocate.com//beer/profile/160/508425/


  2%|▏         | 2/96 [00:13<10:37,  6.79s/it]

...

 98%|█████████▊| 94/96 [10:10<00:12,  6.10s/it]

/beer/profile/716/2779/
https://www.beeradvocate.com//beer/profile/716/2779/


 99%|█████████▉| 95/96 [10:16<00:06,  6.12s/it]

/beer/profile/9262/16764/
https://www.beeradvocate.com//beer/profile/9262/16764/


100%|██████████| 96/96 [10:22<00:00,  6.49s/it]


In [13]:
temp1 = pd.DataFrame(list(zip(mnf, style, alc, ratings, avg_rating, number_of_reviews)), columns = ["mnf","style","alc", "ratings", "avg_rating", "number_of_reviews"])

In [44]:
temp1['pro_link'] =mnfac.pro_link
temp1['beer']=mnfac['product']

In [19]:
def webstite_scrape_p3 (beer_link, number_of_reviews):

    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    url = 'https://www.beeradvocate.com/community/login/'
    driver.get(url)
    
    login_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dd//input[@name ="login"]')))
    login_element.send_keys('suzie102')

    pw_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit"]//dd//ul//li[@id = "ctrl_pageLogin_registered_Disabler"]//input[@name ="password"]')))
    pw_element.send_keys('Beer@addiction102')

    page_click = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit submitUnit"]//dd//input[@type ="submit"]')))
    page_click.click()

    detail_rating = []
    reviews =[]
    avg_per_user =[]
    for link, i in tqdm(zip(beer_link, number_of_reviews)): 
        for pages_i in range(0,int(i)+1,25): #site shows 25 resulst/page)        
            new_url = "https://www.beeradvocate.com"+link+'?view=beer&sort=&start=%d' %(pages_i)
            driver.get(new_url)
            #print(driver.find_element_by_name("hideRatings").is_selected())
            #check_box = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH, '//form[@style="display:inline;margin:0;padding:0;"]//input[@type = "checkbox"]')))#check_box.click()
            #check_box.click()
            time.sleep(5)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source,'html.parser')
        
            detail_rating_i = [ i.get_text() for i in soup.find_all('span', class_ = "muted")][8::3]
            detail_rating.append(detail_rating_i)
            
            reviews_i = [ i.get_text() for i in soup.find_all('div')]
            reviews.append(reviews_i)
            
            avg_i = [i.get_text() for i in soup.find_all('span', class_= "BAscore_norm")]
            print(new_url)
            print(avg_i)
            avg_per_user.append(avg_i)
    return detail_rating, reviews, avg_per_user

In [20]:
temp1.number_of_reviews =[ i for m in number_of_reviews for i in m]
temp1.number_of_reviews = temp1.number_of_reviews.str.replace(",","")

In [38]:
temp1.mnf = temp1.mnf.apply(lambda x: x[-1] if len(x)>1 else x)

In [27]:
temp1.to_csv('temp1.csv')

In [22]:
rate, reviews, avg_user =webstite_scrape_p3 (temp1.pro_link, temp1.number_of_reviews)

0it [00:00, ?it/s]

https://www.beeradvocate.com/beer/profile/713/2124/?view=beer&sort=&start=0
['3.38', '3.83', '3.44', '2.44', '2.55', '2.55', '1.35', '3.28', '2.25', '3.38', '3', '3.22', '2.92', '2.81', '2.83', '2.49', '2.44', '3', '2.25', '3.12', '3', '3.4', '3.1', '2.54', '2.71']
https://www.beeradvocate.com/beer/profile/713/2124/?view=beer&sort=&start=25
['2.94', '2.06', '3.31', '2.11', '3.56', '2.53', '3.05', '2.7', '3.08', '2.66', '2.57', '2.08', '1.92', '2.47', '1.94', '2.65', '1.75', '2.48', '2.62', '2.68', '1.92', '2.99', '3.42', '3', '2.72']
https://www.beeradvocate.com/beer/profile/713/2124/?view=beer&sort=&start=50
['2.55', '2.55', '3.25', '2.44', '2.11', '2.71', '2.24', '3.46', '2.28', '2.35', '2.01', '2.76', '1.84', '2.47', '2.28', '2.74', '2.65', '1.73', '2.83', '2.78', '1.53', '2.85', '2.44', '2.53', '3.47']
https://www.beeradvocate.com/beer/profile/713/2124/?view=beer&sort=&start=75
['2.12', '3.05', '2.53', '2.61', '2.58', '4', '2.29', '2.95', '2.9', '2.89', '2.72', '1.88', '3.6', '1.9'

1it [00:38, 38.97s/it]

https://www.beeradvocate.com/beer/profile/713/2124/?view=beer&sort=&start=125
['3.1', '2.35', '1.92', '2.4', '1.9', '1.8', '2.18', '2.57', '2.5', '1.68', '2.53', '2.53', '2.03', '2.06', '3.1']


2it [00:44, 28.97s/it]

...

https://www.beeradvocate.com/beer/profile/401/4695/?view=beer&sort=&start=200
['2.37', '2.71', '3.77', '1.26', '3.26', '3', '2.57', '3', '3.08', '3.95', '3.45', '3.4', '3', '2.47', '2.95']
https://www.beeradvocate.com/beer/profile/716/2779/?view=beer&sort=&start=0
['4.25', '4.26', '3.94', '3.29', '3.89', '4', '3.4', '4', '3.8', '3.58', '3.84', '3.08', '2.96', '3.22', '3.64', '4.06', '3.61', '4.28', '3.6', '4.17', '3.28', '3.08', '3.71', '2.88', '2.97']
https://www.beeradvocate.com/beer/profile/716/2779/?view=beer&sort=&start=25
['4.48', '5', '4', '2.96', '3.06', '3.61', '3.82', '2.66', '3.11', '2.75', '2.75', '4.83', '2.24', '3.19', '2.79', '3.53', '3.19', '4.47', '3.03', '3.71', '3.15', '3.16', '2.8', '3', '3']
https://www.beeradvocate.com/beer/profile/716/2779/?view=beer&sort=&start=50
['3.08', '5', '2.88', '2.83', '3.33', '3.29', '3.93', '3.76', '3.29', '2.98', '2.93', '2.25', '2.27', '2.89', '3.51', '2.76', '3', '3.23', '3.27', '3.09', '1.87', '3.29', '3.58', '1.03', '4.31']
https:

95it [3:48:20, 94.15s/it]

https://www.beeradvocate.com/beer/profile/716/2779/?view=beer&sort=&start=575
['2.21', '3.92', '3.48', '2.2', '3', '2.82', '3.25', '2.2', '3.05', '2.67', '2.1', '2.97', '3.1', '2.28']
https://www.beeradvocate.com/beer/profile/9262/16764/?view=beer&sort=&start=0
['3.25', '3.26', '2.26', '3.45', '2.75', '3.2', '4', '3.1', '3.27', '3.69', '3.77', '3.39', '2.81', '2.86', '3.49', '3.49', '3', '3.75', '3.33', '3.24', '3.48', '2.71', '3.06', '3.52', '3.11']


96it [3:48:32, 142.84s/it]

https://www.beeradvocate.com/beer/profile/9262/16764/?view=beer&sort=&start=25
['3.58', '3.23', '2.71', '3.41', '3.78', '3.33', '3.53', '2.02', '3.1', '3.53', '3.46', '3.2', '3.21', '2.56', '3.4', '3.31', '3.11', '3.23', '3.53', '3.62', '3', '3.38', '3.03', '3.73']


In [24]:
print([len(i) for i in [rate, reviews, avg_user]])

[2328, 2328, 2328]


In [175]:
temp2 = pd.DataFrame(list(zip(rate, reviews, avg_user )), columns = ['detail_rating', 'reviews', 'avg_per_user'])

In [176]:
#temp2.to_csv('temp2.csv', index = False)

In [177]:
temp2

,detail_rating,reviews,avg_per_user
0,[look: 3.75 | smell: 3.25 | taste: 3.5 | feel:...,[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[3.38, 3.83, 3.44, 2.44, 2.55, 2.55, 1.35, 3.2..."
1,"[Jan 12, 2014, Nov 25, 2013, Sep 22, 2013, Jun...",[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[2.94, 2.06, 3.31, 2.11, 3.56, 2.53, 3.05, 2.7..."
2,"[Jul 04, 2011, Jun 19, 2011, May 23, 2011, May...",[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[2.55, 2.55, 3.25, 2.44, 2.11, 2.71, 2.24, 3.4..."
3,"[Apr 28, 2010, Apr 02, 2010, Feb 05, 2010, Jan...",[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[2.12, 3.05, 2.53, 2.61, 2.58, 4, 2.29, 2.95, ..."
4,"[Feb 11, 2008, Dec 16, 2007, Dec 11, 2007, Sep...",[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[3, 3.03, 3.15, 1.48, 2.57, 2.16, 3.68, 2.98, ..."
...,...,...,...
2323,"[Dec 12, 2004, Dec 08, 2004, Nov 16, 2004, Nov...",[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[3.4, 2.2, 2.38, 3.09, 3.4, 2.8, 2.43, 3, 2.42..."
2324,"[Jan 25, 2004, Dec 30, 2003, Dec 25, 2003, Dec...",[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[1.88, 1.78, 2.15, 2.83, 3.23, 2.73, 2.06, 3.2..."
2325,[look: 3 | smell: 2 | taste: 2 | feel: 2.5 | o...,[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[2.21, 3.92, 3.48, 2.2, 3, 2.82, 3.25, 2.2, 3...."
2326,[look: 3.25 | smell: 3.25 | taste: 3.25 | feel...,[\n\n\n\n\n\n\nHome\n\n\n\nAbout Us\nAdvertise...,"[3.25, 3.26, 2.26, 3.45, 2.75, 3.2, 4, 3.1, 3...."


In [178]:
name=[]
mnf =[]

for mnf_i,beer, link, i in zip(temp1.mnf, temp1.beer, temp1.pro_link, temp1.number_of_reviews): 
    for pages_i in range(0,int(i)+1,25): #site shows 25 resulst/page)
        new_url = link+'?view=beer&sort=&start=%d'%(pages_i)
        name.append(beer)
        mnf.append(mnf_i)
print(len(name), len(mnf))

2328 2328


In [191]:
len(mnf[6])

1

In [180]:
temp2['name'] =name
temp2['mnf']=mnf

In [181]:
def text_cleaning (beerreview):
    text =[]
    for review in tqdm(beerreview):   
        clean_rev = [i.replace("\n","") for i in review[0].split("\xa0") if len(i)>0]
        clean_rev2 =[ re.findall(r'rDev(.*)',i) for i in clean_rev if len(re.findall(r'rDev(.*)',i))>0]
        text.append(clean_rev2)
    return text

In [182]:
def final_clean (text):
    patt="overall:\s*\d\.?\d?\d?"
    clean = [re.findall(patt+"(.*)",i)[0] for i in text if len(re.findall(patt+"(.*)",i))>0]
    return clean

In [183]:
temp2['clean_text'] = text_cleaning(reviews)
temp2['rm_nested_ls'] = temp2['clean_text'].apply(lambda x: [i for m in x for i in m])
temp2['text_reviews'] = temp2['rm_nested_ls'].apply(final_clean)

100%|██████████| 2328/2328 [00:00<00:00, 5935.95it/s]


In [184]:
temp2.drop(['reviews','clean_text','rm_nested_ls'], axis = 1, inplace = True)

In [152]:
temp2.name

0          Yanjing Beer
1          Yanjing Beer
2          Yanjing Beer
3          Yanjing Beer
4          Yanjing Beer
             ...       
2323    Asahi Super Dry
2324    Asahi Super Dry
2325    Asahi Super Dry
2326       Reserva 1925
2327       Reserva 1925
Name: name, Length: 2328, dtype: object

In [197]:
temp2.mnf =temp2.mnf.apply(lambda x: x[0] if len(x)==1 else x)

In [198]:
temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2328 entries, 0 to 2327
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   detail_rating  2328 non-null   object
 1   avg_per_user   2328 non-null   object
 2   name           2328 non-null   object
 3   mnf            2328 non-null   object
 4   text_reviews   2328 non-null   object
dtypes: object(5)
memory usage: 91.1+ KB


In [199]:
temp2['comb'] = temp2.mnf +temp2.name

In [200]:
dt = temp2.groupby('comb').sum()

In [201]:
dt.reset_index(inplace = True)

In [202]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   comb           96 non-null     object
 1   detail_rating  96 non-null     object
 2   avg_per_user   96 non-null     object
 3   name           96 non-null     object
 4   mnf            96 non-null     object
 5   text_reviews   96 non-null     object
dtypes: object(6)
memory usage: 4.6+ KB


In [203]:
temp1

,mnf,style,alc,ratings,avg_rating,number_of_reviews,pro_link,beer,comb
0,Beijing Yanjing Beer Group Corporation,[Lager - European Pale],[4.7%],[250],[2.57],140,/beer/profile/713/2124/,Yanjing Beer,Beijing Yanjing Beer Group CorporationYanjing ...
1,[Wychwood Brewery Company Ltd],[Bitter - English],[4.5%],[0],[0],0,/beer/profile/160/508425/,Hobgoblin Ruby 4.5%,Wychwood Brewery Company LtdHobgoblin Ruby 4.5%
2,Widmer Brothers Brewing Company,[Pale Ale - American],[5.8%],[645],[3.36],128,/beer/profile/8/80302/,Omission Pale Ale (Gluten-Free),Widmer Brothers Brewing CompanyOmission Pale A...
3,Brouwerij Westmalle,[Tripel],[9.5%],"[4,782]",[4.33],1822,/beer/profile/208/646/,Westmalle Trappist Tripel,Brouwerij WestmalleWestmalle Trappist Tripel
4,Wernesgrüner Brauerei AG,[Pilsner - German],[4.9%],[437],[3.44],220,/beer/profile/3966/8653/,Wernesgruner,Wernesgrüner Brauerei AGWernesgruner
...,...,...,...,...,...,...,...,...,...
91,Swinkels Family Brewers,[Lager - European Pale],[5%],[481],[3.02],256,/beer/profile/498/1422/,Bavaria Beer,Swinkels Family BrewersBavaria Beer
92,Bard's Tale Beer Company,[Lager - Adjunct],[4.6%],[136],[2.4],73,/beer/profile/11034/47565/,Bard's Gold,Bard's Tale Beer CompanyBard's Gold
93,Baltika Breweries,[Lager - European / Dortmunder Export],[5.4%],[467],[3.09],215,/beer/profile/401/4695/,Baltika #7 Export,Baltika BreweriesBaltika #7 Export
94,Asahi Breweries Ltd,[Lager - Japanese Rice],[5%],"[1,718]",[2.77],589,/beer/profile/716/2779/,Asahi Super Dry,Asahi Breweries LtdAsahi Super Dry


In [204]:
df3_rv_outsidecan = temp1.merge(dt, on = 'comb')

In [205]:
df3_rv_outsidecan.to_csv('f3_outsideca.csv', index= False)

In [208]:
df3_rv_outsidecan

,mnf_x,style,alc,ratings,avg_rating,number_of_reviews,pro_link,beer,comb,detail_rating,avg_per_user,name,mnf_y,text_reviews
0,Beijing Yanjing Beer Group Corporation,[Lager - European Pale],[4.7%],[250],[2.57],140,/beer/profile/713/2124/,Yanjing Beer,Beijing Yanjing Beer Group CorporationYanjing ...,[look: 3.75 | smell: 3.25 | taste: 3.5 | feel:...,"[3.38, 3.83, 3.44, 2.44, 2.55, 2.55, 1.35, 3.2...",Yanjing BeerYanjing BeerYanjing BeerYanjing Be...,Beijing Yanjing Beer Group CorporationBeijing ...,[12oz bottle from Brewers Haven. Very pale gol...
1,[Wychwood Brewery Company Ltd],[Bitter - English],[4.5%],[0],[0],0,/beer/profile/160/508425/,Hobgoblin Ruby 4.5%,Wychwood Brewery Company LtdHobgoblin Ruby 4.5%,[],[],Hobgoblin Ruby 4.5%,Wychwood Brewery Company Ltd,[]
2,Widmer Brothers Brewing Company,[Pale Ale - American],[5.8%],[645],[3.36],128,/beer/profile/8/80302/,Omission Pale Ale (Gluten-Free),Widmer Brothers Brewing CompanyOmission Pale A...,[look: 3.5 | smell: 3.75 | taste: 3.75 | feel:...,"[3.81, 3.23, 3.36, 3.3, 3.4, 2.6, 3.47, 3.34, ...",Omission Pale Ale (Gluten-Free)Omission Pale A...,Widmer Brothers Brewing CompanyWidmer Brothers...,[From the bottle it pours a surprising amber a...
3,Brouwerij Westmalle,[Tripel],[9.5%],"[4,782]",[4.33],1822,/beer/profile/208/646/,Westmalle Trappist Tripel,Brouwerij WestmalleWestmalle Trappist Tripel,[look: 4 | smell: 4.25 | taste: 4.5 | feel: 4 ...,"[4.31, 4.27, 4.33, 3.43, 4.59, 4.38, 4.38, 4.0...",Westmalle Trappist TripelWestmalle Trappist Tr...,Brouwerij WestmalleBrouwerij WestmalleBrouweri...,"[Westmalle Trappist 'Tripel' @ 9.5% , served f..."
4,Wernesgrüner Brauerei AG,[Pilsner - German],[4.9%],[437],[3.44],220,/beer/profile/3966/8653/,Wernesgruner,Wernesgrüner Brauerei AGWernesgruner,[look: 3.25 | smell: 3.25 | taste: 3.5 | feel:...,"[3.33, 3.92, 4.18, 4.36, 4.14, 3.43, 4.03, 3.3...",WernesgrunerWernesgrunerWernesgrunerWernesgrun...,Wernesgrüner Brauerei AGWernesgrüner Brauerei ...,[Pours a crystal golden color with a pure whit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Swinkels Family Brewers,[Lager - European Pale],[5%],[481],[3.02],256,/beer/profile/498/1422/,Bavaria Beer,Swinkels Family BrewersBavaria Beer,[look: 5 | smell: 5 | taste: 5 | feel: 5 | ove...,"[5, 2.44, 2.55, 3.33, 3.15, 3.68, 1.61, 1.36, ...",Bavaria BeerBavaria BeerBavaria BeerBavaria Be...,Swinkels Family BrewersSwinkels Family Brewers...,[Look this costs one pound a 500 ml bottle in ...
92,Bard's Tale Beer Company,[Lager - Adjunct],[4.6%],[136],[2.4],73,/beer/profile/11034/47565/,Bard's Gold,Bard's Tale Beer CompanyBard's Gold,[look: 1.5 | smell: 1.75 | taste: 1.5 | feel: ...,"[1.56, 2.55, 2.01, 4.42, 4.06, 1.48, 4.08, 3.3...",Bard's GoldBard's GoldBard's Gold,Bard's Tale Beer CompanyBard's Tale Beer Compa...,"[DATE: March 28, 2020.... OCCASION: two weeks ..."
93,Baltika Breweries,[Lager - European / Dortmunder Export],[5.4%],[467],[3.09],215,/beer/profile/401/4695/,Baltika #7 Export,Baltika BreweriesBaltika #7 Export,[look: 4 | smell: 3.25 | taste: 3 | feel: 3.25...,"[3.2, 3.7, 4.06, 3.29, 3.21, 3.12, 3.8, 1, 4.4...",Baltika #7 ExportBaltika #7 ExportBaltika #7 E...,Baltika BreweriesBaltika BreweriesBaltika Brew...,[15.89oz green bottle produced 28 08 19 BB 28 ...
94,Asahi Breweries Ltd,[Lager - Japanese Rice],[5%],"[1,718]",[2.77],589,/beer/profile/716/2779/,Asahi Super Dry,Asahi Breweries LtdAsahi Super Dry,[look: 4.75 | smell: 4 | taste: 4.25 | feel: 4...,"[4.25, 4.26, 3.94, 3.29, 3.89, 4, 3.4, 4, 3.8,...",Asahi Super DryAsahi Super DryAsahi Super DryA...,Asahi Breweries LtdAsahi Breweries LtdAsahi Br...,[When i picked this up i didnt think much of i...
